## 作业-隐式反馈的推荐系统

本次作业中，我们使用对前面的LightGCN进行如下两个改动:
- 用稀疏矩阵的形式构建Movielens图
- 利用隐式反馈实现BPR损失函数
- 不依赖PyG直接实现LightGCN

课程里我们用到的Movielens数据集有明确的用户评分，这是用户的显式反馈。我们训练推荐模型的目标是准确预测用户对他们观看的电影的评分。这种关注评分的方式忽略了考虑用户首先选择观看的电影的重要性，并且没有考虑用户没有评分的电影。缺失的评分更可能是负面的，因为用户往往会先对电影进行一次筛选，再去要观看、评分。选择电影时，用户往往只会观看你认为你会喜欢的电影，而不会去观看他们认为会讨厌的电影。这导致用户只会提交一开始他们希望喜欢的事物的评分，而不会评价不会喜欢的事物。这些没有被评分反映出来的信息，被称为隐式反馈（implicit feedback）。

## 1. 用稀疏矩阵的形式构建Movielens图

首先我们把课件里的部分代码摘抄过来。

In [1]:
import torch
import pandas as pd
df = pd.read_csv('data/MovieLens/raw/ml-latest-small/ratings.csv') 

在实践课里面，我们有用户的评分数据，我们过滤掉评分小于3的数据(去掉低评分)，来构造一个隐式反馈的数据集。

In [2]:
df = df[df.rating >= 3]
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [3]:
user_mapping = {idx: i for i, idx in enumerate(df['userId'].unique())}
movie_mapping = {idx: i for i, idx in enumerate(df['movieId'].unique())}
num_users = len(user_mapping)
num_movies = len(movie_mapping)
print('用户数量:', num_users, '物品数量:', num_movies)

用户数量: 609 物品数量: 8452


In [4]:
user_src = [user_mapping[idx] for idx in df['userId']] # 起始节点
movie_dst = [movie_mapping[idx]+num_users for idx in df['movieId']] # 终止节点
edge_index = torch.tensor([user_src, movie_dst])

划分训练集、验证集和测试集。

In [5]:
_N = edge_index.shape[1]
indicies_perm = torch.randperm(_N)
idx_train = indicies_perm[: int(0.8*_N)]
train_edge_index = edge_index[:, idx_train]

idx_val = indicies_perm[int(0.8*_N): int(0.9*_N)]
val_edge_index = edge_index[:, idx_val]

idx_test = indicies_perm[int(0.9*_N): ]
test_edge_index = edge_index[:, idx_test]

In [6]:
def to_undirected(edge_index):
    edge_index_rev = torch.stack([edge_index[1], edge_index[0]]) # 反向边
    edge_index_sym = torch.cat([edge_index, edge_index_rev], dim=1)
    return edge_index_sym

train_graph_edge_index = to_undirected(train_edge_index)
test_graph_edge_index = to_undirected(torch.cat([train_edge_index, val_edge_index], dim=1))

**完成下面的代码填空（之前的课程里有讲过）**

In [7]:
import scipy.sparse as sp
import numpy as np

def construct_sparse_matrix(data, row, col, N):
    """
    参数说明：
    ---
    data:稀疏矩阵的元素
    row: 非零元素的行序号
    col: 非零元素的列序号
    N: 矩阵的维度大小
    """
    return sp.csr_matrix((data, (row, col)), shape=(N, N))

def normalize_adj(mx):
    """标准化：A' = (D)^-1/2 * ( A ) * (D)^-1/2
    """
    ###################
    #######代码填空#####  
    ################## 

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """将scipy.sparse形式的稀疏矩阵变成torch里的sparse tensor，
       我们需要得到三个变量，row, col和data，它们表示的其实就是
       edge_index和edge_weight。
    """
    ###################
    #######代码填空#####  
    ################## 

In [8]:
def get_adj_norm(row, col, N):
    """
    参数说明：
    ---
    row: 非零元素的行序号
    col: 非零元素的列序号
    N: 矩阵的维度大小
    """
    vals = np.array([1]*len(row))
    adj = construct_sparse_matrix(vals, row, col, N)
    adj_norm = normalize_adj(adj)
    adj_norm = sparse_mx_to_torch_sparse_tensor(adj_norm)
    return adj_norm

In [9]:
train_adj_norm = get_adj_norm(train_graph_edge_index[0], train_graph_edge_index[1], N=num_movies+num_users)
train_adj_norm

<ipython-input-7-4fab2049d4f5>:19: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1/2).flatten() # 得到度矩阵D


tensor(indices=tensor([[ 609,  611,  612,  ...,  608,  608,  608],
                       [   0,    0,    0,  ..., 9056, 9057, 9060]]),
       values=tensor([0.0059, 0.0086, 0.0060,  ..., 0.0331, 0.0331, 0.0331]),
       size=(9061, 9061), nnz=130820, layout=torch.sparse_coo)

In [10]:
test_adj_norm = get_adj_norm(test_graph_edge_index[0], test_graph_edge_index[1], N=num_movies+num_users)

<ipython-input-7-4fab2049d4f5>:19: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1/2).flatten() # 得到度矩阵D


## 2. 实现BPR损失函数

BPR损失函数经常用于训练推荐系统模型。它鼓励已观察到的节点对的预测值，应该大于那些未被观测到的节点对的预测值。给定节点对的排序分数$\hat{y}_{ui}$，BPR损失函数表示为：

$$L_{\text{BPR}} = - \sum_{u=1}^{M} \sum_{i \in \mathcal{N}_u}
\sum_{j \not\in \mathcal{N}_u} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj})
 + \lambda \vert\vert \textbf{x}^{(0)} \vert\vert^2$$

其中$u$表示用户，$i,j$表示物品，$\lambda$控制$L_2$正则项。实际中我们取BPR的均值来作为最后的损失。


这里我们处理一下评分数据rating。后面我们要用到BPR损失函数，它不是一个分类损失，我们把小于等于3的评分作为负样本，大于3的评分作为正样本。

In [11]:
train_mat = train_edge_index.numpy().T
train_mat_set = set()
for x in train_mat:
    train_mat_set.add((x[0], x[1]))

In [12]:
def bpr_loss(positives, negatives, parameters=None, lambda_reg=0):
    """求BPR损失对于所有用户的平均值"""
    ###################
    #######代码填空#####  
    ################## 

def get_pos_neg_pairs(num_neg=1):
    """返回一个三元组(u, i, j)构成的list，其中u表示用户，
       i表示用户交互过的物体，j表示随机采样的用户没有交互过的物品。
    
    参数说明
    ---
    num_neg: 对于每个(u, i)对，采样多少个负样本（没有交互过的物体），默认值为1
    """
    triplets = []
    for u, i in train_mat:
        for _ in range(num_neg):
            j = np.random.randint(num_movies)
            while (u, j) in train_mat_set:
                j = np.random.randint(num_movies)
            triplets.append([u, i, j])
    return triplets

In [13]:
triplets = get_pos_neg_pairs(num_neg=1)
triplets = torch.LongTensor(triplets)

In [14]:
positives = torch.vstack([triplets[:, 0], triplets[:, 1]]) # 正样本
positives

tensor([[ 413,   18,  166,  ...,  428,  544,  481],
        [2789, 1652, 2556,  ..., 2645, 3265,  898]])

In [15]:
negatives = torch.vstack([triplets[:, 0], triplets[:, 2]]) # 负样本
negatives

tensor([[ 413,   18,  166,  ...,  428,  544,  481],
        [3740, 4973, 1550,  ..., 3637, 3552, 7502]])

## 3. 不依赖PyG直接实现LightGCN

在这一节，我们将LightGCN中关于PyG的部分改写为接受稀疏矩阵。

In [16]:
class LGConv(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, adj):
        """前向传播，聚合邻居的信息"""
        return torch.spmm(adj, x)

In [17]:
from torch.nn import ModuleList, Embedding, Linear, ReLU
import torch.nn.functional as F

class LightGCN(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_layers):
        super().__init__()

        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        alpha = 1. / (num_layers + 1)
        self.alpha = torch.tensor([alpha] * (num_layers + 1))

        self.embedding = Embedding(num_nodes, embedding_dim)
        self.convs = ModuleList([LGConv() for _ in range(num_layers)])
        self.decoder = torch.nn.Sequential(Linear(2 * embedding_dim, embedding_dim), 
                                           ReLU(), Linear(embedding_dim, 1))
        self.reset_parameters()

    def reset_parameters(self):
        self.embedding.reset_parameters()

    def get_embedding(self, adj):
        ###################
        #######代码填空#####  
        ################## 
    
    def forward(self, adj, edge_label_index):
        """计算节点对的分数: 对于给定节点对，它们的向量的内积就是其分数"""
        ###################
        #######代码填空#####  
        ################## 

    def predict(self, adj, src_index, dst_index):
        r"""预测给定用户对给定物品的结果。
        
        参数说明
        ----
        src_index: 用户节点的序号。
        dst_index: 待推荐的物品节点的序号。
        """
        ###################
        #######代码填空#####  
        ################## 

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_nodes = num_users + num_movies
model = LightGCN(num_nodes=num_nodes, embedding_dim=32, num_layers=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [19]:
from sklearn.metrics import roc_auc_score

def average_roc_auc(model, adj):
    """对每个用户计算AUC并求平均"""
    user_auc_scores = []
    for user_id in range(num_users):
        pos_item_train_u = pos_item_train[user_id]
        pos_item_test_u = pos_item_test[user_id]
        
        # 去除已经在训练集里的物品
        all_item_ids = np.arange(num_users, num_users + num_movies)
        items_to_rank = np.setdiff1d(all_item_ids, pos_item_train_u)
        
        # 对于每个在pos_item_test_u返回1，否则返回0
        expected = np.in1d(items_to_rank, pos_item_test_u)
        
        if np.sum(expected) >= 1:
            pred = model.predict(adj, user_id, items_to_rank).detach()
            user_auc_scores.append(roc_auc_score(expected, pred))
    return sum(user_auc_scores) / len(user_auc_scores)

In [20]:
from collections import defaultdict
def get_pos_item(edge_index):
    """创建一个字典，字典里存储了每个用户交互过的物品list"""
    pos_item = defaultdict(list)
    for x, y in edge_index.numpy().T:
        pos_item[x].append(y)
    return pos_item

In [21]:
pos_item_train = get_pos_item(train_edge_index)
pos_item_test = get_pos_item(test_edge_index)

In [23]:
def train():
    model.train()
    optimizer.zero_grad()
    out_pos = model(train_adj_norm, positives)
    out_neg = model(train_adj_norm, negatives)
    loss = bpr_loss(out_pos, out_neg, model.embedding.weight, lambda_reg=1e-4)
    loss.backward()
    optimizer.step()
    return float(loss)

positives, negatives = positives.to(device), negatives.to(device) 


print('随机初始化的模型的AUC：', average_roc_auc(model, test_adj_norm))

for epoch in range(1, 301):
    loss = train()
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.5f}')
        
print('训练后的模型的AUC：', average_roc_auc(model, test_adj_norm))

随机初始化的模型的AUC： 0.5472788783405869
Epoch: 020, Loss: 0.00042
Epoch: 040, Loss: 0.00039
Epoch: 060, Loss: 0.00035
Epoch: 080, Loss: 0.00033
Epoch: 100, Loss: 0.00030
Epoch: 120, Loss: 0.00028
Epoch: 140, Loss: 0.00025
Epoch: 160, Loss: 0.00023
Epoch: 180, Loss: 0.00021
Epoch: 200, Loss: 0.00020
Epoch: 220, Loss: 0.00018
Epoch: 240, Loss: 0.00017
Epoch: 260, Loss: 0.00015
Epoch: 280, Loss: 0.00014
Epoch: 300, Loss: 0.00013
训练后的模型的AUC： 0.7466062715954034
